In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import process_data
from script import geolocaliser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Ce premier importe la base "Demande de valeurs foncières" (DVF)

In [3]:
# url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20241008-071041/valeursfoncieres-2023.txt.zip"

url = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/full.csv.gz"
# Envoyer une requête HTTP pour obtenir le fichier CSV

downloaded_file = "full.csv.gz"
response = rq.get(url)

with open(downloaded_file, 'wb') as file:
    file.write(response.content)

# Décompresser le fichier
with gzip.open(downloaded_file, 'rb') as f_in:
    with open("full.csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Charger le fichier CSV dans un DataFrame
df = pd.read_csv("full.csv",encoding="utf-8")

## Affiche la première ligne et et toutes les variables
premiere_ligne = df.iloc[0:9].T
print(premiere_ligne)

# Optionnel : supprimer le fichier compressé après décompression
os.remove("full.csv.gz")
os.remove("full.csv")

C:\Users\flori\AppData\Local\Temp\ipykernel_17180\904652215.py:18: DtypeWarning: Columns (8,10,12,14,17,18,20,22,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv",encoding="utf-8")


                                                0                    1  \
id_mutation                                2023-1               2023-1   
date_mutation                          2023-01-05           2023-01-05   
numero_disposition                              1                    1   
nature_mutation                             Vente                Vente   
valeur_fonciere                         1070000.0            1070000.0   
adresse_numero                              184.0                159.0   
adresse_suffixe                               NaN                  NaN   
adresse_nom_voie                   ALL DES HETRES       ALL DES HETRES   
adresse_code_voie                            0124                 0124   
code_postal                                1630.0               1630.0   
code_commune                                 1354                 1354   
nom_commune                   Saint-Genis-Pouilly  Saint-Genis-Pouilly   
code_departement                      

# Première analyse : afficher toutes les variables, leurs types et le pourcentage des transactions non renseignées

In [4]:
# Création d'un DataFrame avec les noms des colonnes et leurs types
column_types = pd.DataFrame({
    'Nom de la colonne': df.columns,
    'Type': df.dtypes
})

# Affichage du DataFrame avec les noms de colonnes et leurs types
print(column_types)

# Nombre total de transactions réalisées en France
total_transactions = df.shape[0]

# Nombre de transactions manquantes dans la colonne 'valeur_fonciere'
nan_transactions = df['valeur_fonciere'].isna().sum()

# Calcul du pourcentage de valeurs manquantes
pourcentage_nan = (nan_transactions / total_transactions) * 100

# Affichage de la phrase
print(f"En 2023, il y a {pourcentage_nan:.2f}% des {total_transactions} transactions réalisées en France qui ne sont pas renseignées dans la base DVF.")

                                         Nom de la colonne     Type
id_mutation                                    id_mutation   object
date_mutation                                date_mutation   object
numero_disposition                      numero_disposition    int64
nature_mutation                            nature_mutation   object
valeur_fonciere                            valeur_fonciere  float64
adresse_numero                              adresse_numero  float64
adresse_suffixe                            adresse_suffixe   object
adresse_nom_voie                          adresse_nom_voie   object
adresse_code_voie                        adresse_code_voie   object
code_postal                                    code_postal  float64
code_commune                                  code_commune   object
nom_commune                                    nom_commune   object
code_departement                          code_departement   object
ancien_code_commune                    ancien_co

# Premiers filtres
1) Suppression de toutes les colonnes non utilisées
2) Suppression de toutes les transactions non renseignées
3) Reconstitution des adresses complètes

4) Retenir toutes les transactions qui correspondent à des ventes
5) Géolocaliser toutes les transactions qui ne le sont pas dans la base originelle
6) Renseigner les départements et régions

7) On ne garde que les biens particuliers (appartements et maisons)
8) Vérifier que la surface est renseignée
9) Filtre sur les seuls appartements
10) Exclure les biens dans les DROM

In [5]:
## Première opération : alléger le dataset

colonnes_a_supprimer = ['id_mutation',
                        'adresse_suffixe',
                        'id_parcelle',
                        'numero_disposition',
                        'code_nature_culture',
                        'ancien_code_commune',
                        'ancien_nom_commune',
                        'ancien_id_parcelle',
                        'numero_volume',
                        'code_nature_culture_speciale',
                        'nature_culture_speciale',
                        'lot1_numero',
                        'lot2_numero',
                        'lot3_numero',
                        'lot4_numero',
                        'lot5_numero',
                        'lot1_surface_carrez',
                        'lot2_surface_carrez',
                        'lot3_surface_carrez',
                        'lot4_surface_carrez',
                        'lot5_surface_carrez'
                        ]

df.drop(columns=colonnes_a_supprimer, inplace=True)

In [ ]:
# URL de base pour les fichiers DVF
base_url = "https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/full.csv.gz"

# Liste des années
annees = range(2019, 2024)

# Colonnes à supprimer
colonnes_a_supprimer = [
    'id_mutation', 'adresse_suffixe', 'id_parcelle', 'numero_disposition',
    'code_nature_culture', 'ancien_code_commune', 'ancien_nom_commune',
    'ancien_id_parcelle', 'numero_volume', 'code_nature_culture_speciale',
    'nature_culture_speciale', 'lot1_numero', 'lot2_numero', 'lot3_numero',
    'lot4_numero', 'lot5_numero', 'lot1_surface_carrez', 'lot2_surface_carrez',
    'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez'
]

KeyboardInterrupt: 

In [ ]:
dataframes = []

# Boucle pour traiter tous les fichiers
for annee in annees:
    df = process_data.traiter_fichier(annee, base_url, colonnes_a_supprimer)
    dataframes.append(df)

df_concatene = pd.concat(dataframes, ignore_index=True)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_4656\773889705.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4656\773889705.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4656\773889705.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_4656\773

In [ ]:
# Convertir les colonnes spécifiques en chaînes de caractères
colonnes_a_convertir = ['adresse_code_voie', 'code_commune', 'code_departement']
df_concatene = process_data.convertir_en_str(df_concatene, colonnes_a_convertir)

In [ ]:
df_concatene.to_parquet(
    "data/dvf.parquet",
    index=False,
    compression="gzip",
    engine="pyarrow",
    use_dictionary=True  # Active l'encodage par dictionnaire
)

In [6]:
# 2ème opération (filtre) : Supprimer toutes les transactions NaN
df = df.dropna(subset=['valeur_fonciere'])

print(df['valeur_fonciere'].isna().sum())

0


In [7]:
# 3ème opération : conversion des adresses en string

print(df['adresse_numero'].dtype)
print(df['adresse_nom_voie'].dtype)
print(df['code_postal'].dtype)
print(df['nom_commune'].dtype)

colonnes_a_nettoyer = ['adresse_numero', 'code_postal']
df = process_data.nettoyer_colonnes(df, colonnes_a_nettoyer)
# Convertir la colonne 'code_commune' en type string
df['code_commune'] = df['code_commune'].astype('string')

# Ajouter un '0' au début si la chaîne a 4 caractères
df['code_commune'] = [x.zfill(5) if len(x) == 4 else x for x in df['code_commune']]

# Vérifier les résultats
print(df['code_commune'].head())

float64
object
float64
object
0    01354
1    01354
2    01354
3    01404
4    01361
Name: code_commune, dtype: object


In [8]:
# 4ème opération (si besoin de géolocalisation) : Compléter par le type de voie

voie = pd.read_csv("data/voie.csv",sep=";",encoding="utf-8")
print(voie.head())

# Liste des abréviations de types de voie
abbreviations = voie['abreviation'].tolist()

# Appliquer la fonction à la colonne 'adresse_nom_voie'
result = [process_data.check_abbreviation(adresse) for adresse in df['adresse_nom_voie']]

# Décomposer les résultats dans les colonnes 'type_voie' et 'nom_voie'
df['type_voie'] = [x[0] for x in result]
df['nom_voie'] = [x[1] for x in result]

df = df.merge(voie,left_on=['type_voie'],right_on=['abreviation'])

  abreviation type_voie_complet
0         RUE               Rue
1          AV            Avenue
2         RTE             Route
3         CHE            Chemin
4          BD         Boulevard


TypeError: check_abbreviation() missing 1 required positional argument: 'abbreviations'

In [ ]:
## 5ème : Réécriture de l'adresse

df['Adresse'] = df['adresse_numero'] + ' ' + df['type_voie_complet'] + ' ' + df['nom_voie'] + ' ' + df['code_postal'] + ' ' + df['nom_commune']

print(df['Adresse'].head())

0       184 Allée DES HETRES 1630 Saint-Genis-Pouilly
1       159 Allée DES HETRES 1630 Saint-Genis-Pouilly
2       159 Allée DES HETRES 1630 Saint-Genis-Pouilly
3         2914 Route DE PONCIN 1450 Serrières-sur-Ain
4    427 Chemin DE L'AUBEPIN 1800 Saint-Jean-de-Niost
Name: Adresse, dtype: string


In [ ]:
# Nombre total de lignes dans df_geolocalisees2
nb_lignes_total = len(df)
print(f"Nombre total de lignes dans df_geolocalisees2 : {nb_lignes_total}")

# Compter les modalités de 'nature_mutation' et calculer les pourcentages
modalites_nature_mutation = df['nature_mutation'].value_counts()
modalites_nature_mutation_percent = (modalites_nature_mutation / nb_lignes_total * 100).round(1)

# Combiner les valeurs et les pourcentages dans un DataFrame
tableau_nature_mutation = pd.DataFrame({
    'Nombre de transactions': modalites_nature_mutation,
    'Pourcentage (%)': modalites_nature_mutation_percent
})

# Afficher le tableau
print("\nTableau des modalités de 'nature_mutation' avec pourcentages :")
print(tableau_nature_mutation)


Nombre total de lignes dans df_geolocalisees2 : 2262335

Tableau des modalités de 'nature_mutation' avec pourcentages :
                                    Nombre de transactions  Pourcentage (%)
nature_mutation                                                            
Vente                                              2126208             94.0
Vente en l'état futur d'achèvement                  116294              5.1
Echange                                               8922              0.4
Adjudication                                          6541              0.3
Vente terrain à bâtir                                 4200              0.2
Expropriation                                          170              0.0


In [ ]:
# Filtre : on ne conserve que les ventes
df = df[df['nature_mutation']=='Vente']

In [ ]:
df = df.drop(columns=['nature_mutation'])

In [ ]:

df_geolocalisees = df

In [ ]:
# Charger les données des codes de région
region = pd.read_csv('data/code_region.csv', sep=';')
print(region['departmentCode'].dtype)
print(df_geolocalisees['code_departement'].dtype)

# Afficher les premières lignes pour vérifier
print(region.head())

# Vérifier le type de la colonne 'departmentCode'
print(f"Type de la colonne 'departmentCode': {region['departmentCode'].dtype}")

# Vérifier les modalités (valeurs uniques) de 'departmentCode'
print("Modalités de 'departmentCode':")
print(region['departmentCode'].unique())
print(df_geolocalisees['code_departement'].unique())

object
object
  departmentCode           departmentName  regionCode  \
0              1                      Ain        84.0   
1              2                    Aisne        32.0   
2              3                   Allier        84.0   
3              4  Alpes-de-Haute-Provence        93.0   
4              5             Hautes-Alpes        93.0   

                   regionName  
0        Auvergne-Rhône-Alpes  
1             Hauts-de-France  
2        Auvergne-Rhône-Alpes  
3  Provence-Alpes-Côte d'Azur  
4  Provence-Alpes-Côte d'Azur  
Type de la colonne 'departmentCode': object
Modalités de 'departmentCode':
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29' '2A' '2B'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75'

In [ ]:
region['departmentCode'] = region['departmentCode'].astype(str)
df_geolocalisees['code_departement'] = df_geolocalisees['code_departement'].astype(str)

print(region['departmentCode'].unique())
print(df_geolocalisees['code_departement'].unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29' '2A' '2B'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '971' '972' '973' '974'
 '976' '987' '988']
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29' '2A' '2B'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '69' '70' '71' '72' '73' '74'
 '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86' '87' '88' '89'
 '90' '91' '92' '93' '94' '95' '971' '972' '973' '9

In [ ]:
#Fusionner df_geolocalisees avec les codes de région
df_geolocalisees = df_geolocalisees.merge(region, left_on=["code_departement"], right_on=['departmentCode'], how='left')

df_geolocalisees['regionCode'] = df_geolocalisees['regionCode'].astype('Int64')  # Utilise Int64 pour gérer les valeurs manquantes
#Compter le nombre de transactions par région

transactions_par_region = df_geolocalisees.groupby('regionName').size()

# Calculer le total des transactions
total_transactions = transactions_par_region.sum()

# Calculer la part de chaque région dans le total des transactions
part_region = (transactions_par_region / total_transactions) * 100

# Créer un DataFrame avec les résultats
tableau_regions = pd.DataFrame({
    'Nombre de transactions': transactions_par_region,
    'Part du total (%)': part_region
})

# Afficher le tableau des régions
print(tableau_regions)

                            Nombre de transactions  Part du total (%)
regionName                                                           
Auvergne-Rhône-Alpes                        273064          12.842770
Bourgogne-Franche-Comté                     106350           5.001862
Bretagne                                    103028           4.845622
Centre-Val de Loire                          94763           4.456902
Corse                                         5306           0.249552
Grand Est                                    88742           4.173721
Guadeloupe                                    4029           0.189492
Guyane                                        2311           0.108691
Hauts-de-France                             173535           8.161713
La Réunion                                   16526           0.777252
Martinique                                    3833           0.180274
Normandie                                   115664           5.439919
Nouvelle-Aquitaine  

In [ ]:
# Retirer les Départements et Régions Outre-Mer
df_geolocalisees = df_geolocalisees[~df_geolocalisees['regionName'].isin(['Guyane', 'Martinique', 'Guadeloupe', 'La Réunion', 'Mayotte'])]


In [ ]:
# Création d'un DataFrame avec les noms des colonnes et leurs types
column_types = pd.DataFrame({
    'Nom de la colonne': df_geolocalisees.columns,
    'Type': df_geolocalisees.dtypes
})

# Affichage du DataFrame avec les noms de colonnes et leurs types
print(column_types)

print(df_geolocalisees.head())

                                   Nom de la colonne            Type
date_mutation                          date_mutation          object
valeur_fonciere                      valeur_fonciere         float64
adresse_numero                        adresse_numero  string[python]
adresse_nom_voie                    adresse_nom_voie          object
adresse_code_voie                  adresse_code_voie          object
code_postal                              code_postal  string[python]
code_commune                            code_commune          object
nom_commune                              nom_commune          object
code_departement                    code_departement          object
nombre_lots                              nombre_lots           int64
code_type_local                      code_type_local         float64
type_local                                type_local          object
surface_reelle_bati              surface_reelle_bati         float64
nombre_pieces_principales  nombre_

In [ ]:
# Calculer la répartition des pièces > 0 par type de bien
repartition_pieces = pd.crosstab(
    index=df_geolocalisees['type_local'], 
    columns=df_geolocalisees['nombre_pieces_principales'] > 0, 
    normalize='index'
) * 100

# Calculer la répartition des surfaces renseignées par type de bien
repartition_surface = pd.crosstab(
    index=df_geolocalisees['type_local'], 
    columns=df_geolocalisees['surface_reelle_bati'].notna(), 
    normalize='index'
) * 100

# Renommer les colonnes pour chaque répartition
repartition_pieces.columns = ['Pas de pièces > 0', 'Pièces > 0']
repartition_surface.columns = ['Surface manquante', 'Surface renseignée']

# Fusionner les deux tables en une seule
repartition_combined = pd.concat([repartition_pieces, repartition_surface], axis=1)

# Arrondir et ajouter le signe %
repartition_combined = repartition_combined.round(1).astype(str) + '%'

# Afficher le tableau combiné
print(repartition_combined)


                                         Pas de pièces > 0 Pièces > 0  \
type_local                                                              
Appartement                                           0.2%      99.8%   
Dépendance                                          100.0%       0.0%   
Local industriel. commercial ou assimilé            100.0%       0.0%   
Maison                                                0.1%      99.9%   

                                         Surface manquante Surface renseignée  
type_local                                                                     
Appartement                                           0.0%             100.0%  
Dépendance                                          100.0%               0.0%  
Local industriel. commercial ou assimilé              3.8%              96.2%  
Maison                                                0.0%             100.0%  


In [ ]:
# One ne garde donc que les maisons et les appartements

df_geolocalisees2 = df_geolocalisees[df_geolocalisees['type_local'].isin(['Maison', 'Appartement'])]

total_transactions = df_geolocalisees.shape[0]
total_transactions2 = df_geolocalisees2.shape[0]

# Calcul du nombre d'appartements et de maisons dans ce sous-ensemble
appartements = df_geolocalisees2[df_geolocalisees['type_local'] == 'Appartement'].shape[0]
maisons = df_geolocalisees2[df_geolocalisees['type_local'] == 'Maison'].shape[0]

# Calcul des pourcentages
pourcentage = total_transactions2/total_transactions * 100
pourcentage_appartements = (appartements / total_transactions2) * 100
pourcentage_maisons = (maisons / total_transactions2) * 100

# Générer la phrase
phrase = f"Sur les {total_transactions} transactions renseignées et géolocalisées, on ne conserve que {total_transactions2} (soit {pourcentage:.1f}%)" \
         f"transactions représentant {pourcentage_appartements:.1f}% d'appartements et {pourcentage_maisons:.1f}% de maisons."

# Afficher la phrase
print(phrase)

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\2367473537.py:9: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\2367473537.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Sur les 2099509 transactions renseignées et géolocalisées, on ne conserve que 954313 (soit 45.5%)transactions représentant 49.3% d'appartements et 50.7% de maisons.


In [ ]:
# Vérifier si la surface est renseignée dans la colonne 'surface_reelle_bati'
surface_renseignée = df_geolocalisees2['surface_reelle_bati'].notna().sum()

# Calcul du nombre total de lignes dans le DataFrame
total_lignes = df_geolocalisees2.shape[0]

# Calcul du pourcentage de valeurs renseignées
pourcentage_surface_renseignée = (surface_renseignée / total_lignes) * 100

# Afficher le résultat
print(f"Pourcentage de lignes avec surface renseignée : {pourcentage_surface_renseignée:.1f}%")


Pourcentage de lignes avec surface renseignée : 100.0%


In [ ]:
# Pour l'instant, on ne garde que les appartements
df_geolocalisees2 = df_geolocalisees2[df_geolocalisees2['type_local'] == 'Appartement']


In [ ]:
df = df.drop(columns=['code_type_local','type_local','surface_terrain'])

In [ ]:
# Calcule le prix moyen par mètre carré
df_geolocalisees2['prix_m2'] = df_geolocalisees2['valeur_fonciere']/df_geolocalisees2['surface_reelle_bati']

In [ ]:
# Problème : certains terrains sont déclarés en habitation
# Filtrer les lignes où 'nature_culture' n'est pas NaN
df_filtered = df_geolocalisees2[df_geolocalisees2['nature_culture'].notna()]

# Sélectionner uniquement les colonnes désirées
df_filtered = df_filtered[['type_local','valeur_fonciere','nature_culture', 'surface_reelle_bati', 'surface_terrain', 'prix_m2']]

# Afficher les 10 premières lignes
print(df_filtered.head(10))

df_geolocalisees2 = df_geolocalisees2[df_geolocalisees2['nature_culture'].isna()]

     type_local  valeur_fonciere nature_culture  surface_reelle_bati  \
16  Appartement         430000.0           sols                 51.0   
17  Appartement         430000.0           sols                 51.0   
19  Appartement         430000.0           sols                 40.0   
24  Appartement         430000.0           sols                 48.0   
25  Appartement         430000.0           sols                 40.0   
32  Appartement          40000.0           sols                 44.0   
33  Appartement          40000.0           sols                 50.0   
59  Appartement        1040000.0           sols                108.0   
60  Appartement        1040000.0           sols                 54.0   
61  Appartement        1040000.0           sols                149.0   

    surface_terrain       prix_m2  
16             89.0   8431.372549  
17             89.0   8431.372549  
19             99.0  10750.000000  
24             99.0   8958.333333  
25             99.0  10750.

In [ ]:
# Une partie des transactions sont en doublon

# Compter les doublons en fonction des colonnes spécifiées
nb_doublons = df_geolocalisees2.duplicated(subset=['adresse_nom_voie', 'code_postal', 'surface_reelle_bati', 'valeur_fonciere']).sum()
print(f"Nombre de doublons : {nb_doublons}")

# Supprimer les doublons en gardant la première occurrence
df_geolocalisees2 = df_geolocalisees2.drop_duplicates(subset=['adresse_nom_voie', 'code_postal', 'surface_reelle_bati', 'valeur_fonciere'], keep='first')

# Vérifier le nouveau nombre de lignes
print(f"Nombre de lignes après suppression des doublons : {len(df_geolocalisees2)}")


Nombre de doublons : 10729
Nombre de lignes après suppression des doublons : 360087


In [ ]:
# Calcul de la moyenne des prix totale en fonction de la surface
prix_moyen_m2 = df_geolocalisees2['valeur_fonciere'].sum() / df_geolocalisees2['surface_reelle_bati'].sum()
print(f"Moyenne totale des prix en fonction de la surface : {prix_moyen_m2:.2f} €")

# Calcul de la moyenne des prix par région
prix_moyen_region = df_geolocalisees2.groupby('regionName')['valeur_fonciere'].sum() / df_geolocalisees2.groupby('regionName')['surface_reelle_bati'].sum()

# Calcul de la moyenne des prix par type d'appartement en fonction de la surface
prix_moyen_type = df_geolocalisees2.groupby('type_local')['valeur_fonciere'].sum() / df_geolocalisees2.groupby('type_local')['surface_reelle_bati'].sum()

# Calcul du nombre de transactions par région
nb_transactions_region = df_geolocalisees2.groupby('regionName').size()

# Calcul du pourcentage d'appartements et de maisons par région
nb_transactions_appart = df_geolocalisees2[df_geolocalisees2['type_local'] == "Appartement"].groupby('regionName').size()
nb_transactions_maison = df_geolocalisees2[df_geolocalisees2['type_local'] == "Maison"].groupby('regionName').size()

# Calculer le pourcentage pour chaque type par région
pourcentage_appart = (nb_transactions_appart / nb_transactions_region * 100).fillna(0)
pourcentage_maison = (nb_transactions_maison / nb_transactions_region * 100).fillna(0)

# Combiner toutes les informations dans un DataFrame final
stats_region = pd.DataFrame({
    'prix_moyen_region': prix_moyen_region,
    'nb_transactions': nb_transactions_region,
    'pourcentage_appart': pourcentage_appart,
    'pourcentage_maison': pourcentage_maison
}).reset_index()

# Afficher le tableau final
print(stats_region)
# Afficher les tableaux de moyennes
print("\nMoyenne des prix par région en fonction de la surface :")
print(prix_moyen_region)

print("\nMoyenne des prix par type d'appartement en fonction de la surface :")
print(prix_moyen_type)


Moyenne totale des prix en fonction de la surface : 4645.11 €
                    regionName  prix_moyen_region  nb_transactions  \
0         Auvergne-Rhône-Alpes        3604.943521            55328   
1      Bourgogne-Franche-Comté        2208.715567            13281   
2                     Bretagne        3217.213380            14376   
3          Centre-Val de Loire        2407.473468             8005   
4                        Corse        3762.884222             1965   
5                    Grand Est        2088.459393            10425   
6              Hauts-de-France        3110.236742            14088   
7                    Normandie        2953.461407            13598   
8           Nouvelle-Aquitaine        3623.930447            24710   
9                    Occitanie        3264.438937            38818   
10            Pays de la Loire        3538.854014            14414   
11  Provence-Alpes-Côte d'Azur        4778.416606            58154   
12               Île-de-Fran

In [ ]:
# URL du fichier GeoJSON des régions de France
url_region = "https://france-geojson.gregoiredavid.fr/repo/regions.geojson"

# Récupérer le fichier GeoJSON des régions de France
response = rq.get(url_region)
regions_geojson = response.json()

# Créer une liste pour stocker les noms des régions
region_names = []

# Extraire les noms des régions depuis les propriétés de chaque feature
for feature in regions_geojson['features']:
    region_name = feature['properties']['nom']
    region_names.append(region_name)

# Créer un DataFrame avec les noms des régions
df_regions = pd.DataFrame(region_names, columns=['regionName'])
    
# Afficher le DataFrame
print(df_regions)

 # Afficher les modalités de la colonne 'regionName'
modalites_region = df_geolocalisees2['regionName'].value_counts().reset_index()

modalites_region = pd.DataFrame(modalites_region[['regionName']])
# Affichage
print(modalites_region)


                    regionName
0                Île-de-France
1          Centre-Val de Loire
2      Bourgogne-Franche-Comté
3                    Normandie
4              Hauts-de-France
5                    Grand Est
6             Pays de la Loire
7                     Bretagne
8           Nouvelle-Aquitaine
9                   Guadeloupe
10                  Martinique
11                      Guyane
12                  La Réunion
13                     Mayotte
14                   Occitanie
15        Auvergne-Rhône-Alpes
16  Provence-Alpes-Côte d'Azur
17                       Corse
                    regionName
0                Île-de-France
1   Provence-Alpes-Côte d'Azur
2         Auvergne-Rhône-Alpes
3                    Occitanie
4           Nouvelle-Aquitaine
5             Pays de la Loire
6                     Bretagne
7              Hauts-de-France
8                    Normandie
9      Bourgogne-Franche-Comté
10                   Grand Est
11         Centre-Val de Loire
12      

## Travail sur les zones cotières
1) Import des listes des communes littorales
2) Calcul des centroïdes des communes (afin d'orienter la visualisation sur la commune)


In [ ]:
# Charger les données des codes de région
dep = pd.read_csv('data/code_region.csv', sep=';')
print(region['departmentName'].dtype)
print(df_geolocalisees2['departmentName'].dtype)


# Vérifier le type de la colonne 'departmentCode'
print(f"Type de la colonne 'departmentCode': {dep['departmentName'].dtype}")

# Vérifier les modalités (valeurs uniques) de 'departmentCode'
print("Modalités de 'departmentCode':")
print(dep['departmentName'].unique())
print(df_geolocalisees2['departmentName'].unique())

object
object
Type de la colonne 'departmentCode': object
Modalités de 'departmentCode':
['Ain' 'Aisne' 'Allier' 'Alpes-de-Haute-Provence' 'Hautes-Alpes'
 'Alpes-Maritimes' 'Ardèche' 'Ardennes' 'Ariège' 'Aube' 'Aude' 'Aveyron'
 'Bouches-du-Rhône' 'Calvados' 'Cantal' 'Charente' 'Charente-Maritime'
 'Cher' 'Corrèze' "Côte-d'Or" "Côtes-d'Armor" 'Creuse' 'Dordogne' 'Doubs'
 'Drôme' 'Eure' 'Eure-et-Loir' 'Finistère' 'Corse-du-Sud' 'Haute-Corse'
 'Gard' 'Haute-Garonne' 'Gers' 'Gironde' 'Hérault' 'Ille-et-Vilaine'
 'Indre' 'Indre-et-Loire' 'Isère' 'Jura' 'Landes' 'Loir-et-Cher' 'Loire'
 'Haute-Loire' 'Loire-Atlantique' 'Loiret' 'Lot' 'Lot-et-Garonne' 'Lozère'
 'Maine-et-Loire' 'Manche' 'Marne' 'Haute-Marne' 'Mayenne'
 'Meurthe-et-Moselle' 'Meuse' 'Morbihan' 'Moselle' 'Nièvre' 'Nord' 'Oise'
 'Orne' 'Pas-de-Calais' 'Puy-de-Dôme' 'Pyrénées-Atlantiques'
 'Hautes-Pyrénées' 'Pyrénées-Orientales' 'Bas-Rhin' 'Haut-Rhin' 'Rhône'
 'Haute-Saône' 'Saône-et-Loire' 'Sarthe' 'Savoie' 'Haute-Savoie' 'Paris'


In [ ]:
# Création de la variable adresse_py en remplaçant les espaces par des "+"
df_geolocalisees2['adresse_py'] = df_geolocalisees2['Adresse'].str.replace(' ', '+', regex=True)

In [ ]:
commune = pd.read_csv('data/communes_code.csv', sep=';',encoding="utf-8")
show(commune)
# Left_join des codes_communes avec les noms des communes geoJson
df_geolocalisees2 = df_geolocalisees2.merge(commune, left_on=["code_commune"], right_on=['code'], how='left')
df_geolocalisees2 = df_geolocalisees2.sort_values(by='code_commune', ascending=True)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_11792\2638450821.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\2638450821.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\2638450821.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11

In [ ]:
df_cotieres = pd.read_csv("data/communes_cotieres.csv",sep=";")
show(df_cotieres.head())

# # Statistiques par "Motif du classement"
# stats_motif = df_cotieres.groupby('Motif du classement').agg(
#     nombre_communes=('Nom commune', 'nunique'),
#     population_totale=('Population', 'sum')
# ).reset_index()

# # Statistiques par "Région"
# stats_region = df_cotieres.groupby('Région').agg(
#     nombre_communes=('Nom commune', 'nunique'),
#     population_totale=('Population', 'sum')
# ).reset_index()

# # Afficher les résultats
# print(stats_motif)
# print(stats_region)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1427992124.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1427992124.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1427992124.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11

In [ ]:
liste_cotieres = df_cotieres['nom'].unique().tolist()

print(len(liste_cotieres))

619


In [ ]:
df_geolocalisees3 = df_geolocalisees2[df_geolocalisees2['communeName'].isin(liste_cotieres)]

In [ ]:
# Combien de transactions ne sont pas géolocalisées ?

# Nombre total de transactions renseignées (celles qui ont des valeurs dans 'latitude' et 'longitude')
total_transactions_renseignees = df_geolocalisees3.shape[0]

# Nombre de lignes où 'latitude' ou 'longitude' est manquant (sans double compte)
non_geolocalisees = df_geolocalisees3[df_geolocalisees3['latitude'].isna() | df_geolocalisees3['longitude'].isna()].shape[0]

# Calcul du pourcentage de transactions non géolocalisées
pourcentage_non_geolocalisees = (non_geolocalisees / total_transactions_renseignees) * 100

# Affichage de la phrase
print(f"Sur les {total_transactions_renseignees} transactions renseignées, {pourcentage_non_geolocalisees:.2f}% ne sont pas géolocalisées.")

Sur les 51115 transactions renseignées, 0.28% ne sont pas géolocalisées.


In [ ]:
# Calculer le nombre de transactions par commune
nombre_transactions_cotieres = df_geolocalisees3.groupby('communeName').size().reset_index(name='Nombre de transactions')

show(nombre_transactions_cotieres)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1770471222.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1770471222.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1770471222.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11

In [ ]:
df_missing = df_geolocalisees2[df_geolocalisees2['latitude'].isna()]

In [ ]:
df_missing = df_missing.drop(columns=['latitude', 'longitude'])

In [ ]:
df_missing = geolocaliser.geolocaliser_actifs(df_missing, 'adresse_py', 'latitude', 'longitude')

In [ ]:
# Concatène les DataFrames en réinitialisant l'index
df_final = pd.concat([df_geolocalisees3, df_missing], ignore_index=True)

In [ ]:
# Affiche le DataFrame final pour vérifier la concaténation

print(df_geolocalisees3.shape)
print(df_missing.shape)
print(df_final.shape)

(51115, 31)
(1195, 31)
(52310, 31)


In [ ]:
# # Extraire la liste des communes uniques
# df_communes = df_final['communeName'].drop_duplicates().dropna().reset_index(drop=True).to_frame()

# # Remplir les coordonnées des mairies
# df_communes = geolocaliser.geolocaliser_mairies(df_communes, 'communeName')

# show(df_communes)

show(df_final.head())

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1743043137.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1743043137.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11792\1743043137.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_11

In [ ]:
# Effectuer un left join entre df_final et df_communes
base_transactions = pd.merge(df_final, df_communes, how='left', left_on='communeName', right_on='communeName')
print(base_transactions.shape[0])
print(base_transactions.shape[1])

# Vérification du résultat
print(base_transactions.head())

52310
33
  date_mutation  valeur_fonciere adresse_numero        adresse_nom_voie  \
0    2023-10-05         147886.0             34           BD ALBERT 1ER   
1    2023-06-02         219000.0            142  CHE AMES DU PURGATOIRE   
2    2023-05-25         480000.0            978       RTE DE SAINT JEAN   
3    2023-05-17         169000.0             15               CHE TANIT   
4    2023-10-06         360000.0              4     AV COMMANDANT GARBE   

  adresse_code_voie code_postal code_commune nom_commune code_departement  \
0              0060        6600        06004     Antibes                6   
1              0110        6600        06004     Antibes                6   
2              3200        6600        06004     Antibes                6   
3              3370        6160        06004     Antibes                6   
4              0770        6160        06004     Antibes                6   

   nombre_lots  ...  departmentCode   departmentName  regionCode  \
0        

In [ ]:
base_transactions = base_transactions.drop(columns=['departmentCode','communeName','code'])

In [ ]:
base_transactions.to_csv("data/base.csv",sep=";",index=False,encoding="utf-8")